## Importing

In [1]:
import argparse
import math
import time

import torch
import yaml
import numpy as np

from audit import get_average_audit_results, audit_models, audit_records, sample_auditing_dataset, exp_estimated_epsilon
from get_signals import get_model_signals
from models.utils import load_models, train_models, split_dataset_for_training
from util import (
    check_configs,
    setup_log,
    initialize_seeds,
    create_directories,
    load_dataset,
    load_subset_dataset,
)

# Enable benchmark mode in cudnn to improve performance when input sizes are consistent
torch.backends.cudnn.benchmark = True

/data/yx23/anaconda3/envs/privacy_meter/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load config

In [2]:
configs = "configs/mnist.yaml"
with open(configs, "rb") as f:
    configs = yaml.load(f, Loader=yaml.Loader)

# Validate configurations
check_configs(configs)

# Initialize seeds for reproducibility
initialize_seeds(configs["run"]["random_seed"])

# Create necessary directories
subdata_dir = configs["run"]["log_dir"] 
log_dir = configs["run"]["log_dir"] + configs["train"]["model_name"] #+ "/" + configs["train"]["method"]
if configs["train"]["epsilon"] > 0:
    log_dir += f"/eps{int(configs["train"]["epsilon"])}"
else:
    log_dir += "/nonpri"
configs["run"]["log_dir"] = log_dir
directories = {
    "log_dir": log_dir,
    "report_dir": f"{log_dir}/report",
    "signal_dir": f"{log_dir}/signals",
    "data_dir": configs["data"]["data_dir"],
    "subdata_dir": subdata_dir,
}
create_directories(directories)

# Set up logger
logger = setup_log(
    directories["report_dir"], "time_analysis", configs["run"]["time_log"]
)

start_time = time.time()

In [3]:
log_dir

'exp/demo_mnist/LR/eps100'

## Load dataset

In [4]:
baseline_time = time.time()
dataset = load_dataset(configs, directories["data_dir"], logger)
logger.info("Loading dataset took %0.5f seconds", time.time() - baseline_time)

2024-12-18 11:20:35,738 INFO     Load data from data/mnist.pkl
12/18/2024 11:20:35:INFO:Load data from data/mnist.pkl
2024-12-18 11:20:35,740 INFO     The whole dataset size: 70000
12/18/2024 11:20:35:INFO:The whole dataset size: 70000
2024-12-18 11:20:35,741 INFO     Loading dataset took 0.03804 seconds
12/18/2024 11:20:35:INFO:Loading dataset took 0.03804 seconds


In [5]:
# subset of dataset
if configs["train"]["data_size"] < len(dataset):
    dataset = load_subset_dataset(configs, dataset, f"{directories["subdata_dir"]}", logger)
    logger.info("Loading sub-dataset took %0.5f seconds", time.time() - baseline_time)

2024-12-18 11:20:38,798 INFO     Load data from exp/demo_mnist/mnist.pkl
12/18/2024 11:20:38:INFO:Load data from exp/demo_mnist/mnist.pkl
2024-12-18 11:20:38,803 INFO     Loading sub-dataset took 3.10018 seconds
12/18/2024 11:20:38:INFO:Loading sub-dataset took 3.10018 seconds


## Load or train models

In [ ]:
# Define experiment parameters
num_experiments = configs["run"]["num_experiments"]
num_reference_models = configs["audit"]["num_ref_models"]
num_model_pairs = max(math.ceil(num_experiments / 2.0), num_reference_models + 1)

# Load or train models
baseline_time = time.time()
models_list, memberships = load_models(
    log_dir, dataset, num_model_pairs * 2, configs, logger
)


In [7]:
if models_list is None:
    # Split dataset for training two models per pair
    data_splits, memberships = split_dataset_for_training(
        len(dataset), num_model_pairs, ratio=0.5
    )
    models_list = train_models(
        log_dir, dataset, data_splits, memberships, configs, logger
    )
logger.info(
    "Model loading/training took %0.1f seconds", time.time() - baseline_time
)

2024-12-18 11:11:36,050 INFO     Model loading/training took 2.9 seconds
12/18/2024 11:11:36:INFO:Model loading/training took 2.9 seconds


## Prepare auditing dataset

In [7]:
# auditing_dataset, auditing_membership = sample_auditing_dataset(
#         configs, dataset, logger, memberships
#     )
auditing_dataset, auditing_membership = dataset, memberships

## Compute signals

In [8]:
baseline_time = time.time()
signals = get_model_signals(models_list, auditing_dataset, configs, logger)
logger.info("Preparing signals took %0.5f seconds", time.time() - baseline_time)

2024-12-18 11:20:46,190 INFO     Signals loaded from disk.
12/18/2024 11:20:46:INFO:Signals loaded from disk.
2024-12-18 11:20:46,191 INFO     Preparing signals took 0.00582 seconds
12/18/2024 11:20:46:INFO:Preparing signals took 0.00582 seconds


## Attack scores

In [9]:
# Perform the privacy audit
baseline_time = time.time()
#target_model_indices = list(range(num_experiments))
target_model_indices = list(range((num_reference_models+1)*2)) # for all pair models

# shape of mia_score_list: (n, m) n=(num_reference_models+1)*2, m=len(auditing_dataset)
mia_score_list, membership_list = audit_records(
    f"{directories['report_dir']}",
    target_model_indices,
    signals,
    auditing_membership,
    num_reference_models,
    logger,
    attack_algorithm=configs["audit"]["algorithm"],
)

2024-12-18 11:20:48,180 INFO     mia_score_list and membership_list loaded to disk.
12/18/2024 11:20:48:INFO:mia_score_list and membership_list loaded to disk.


In [12]:
# if len(target_model_indices) > 1:
#     logger.info(
#         "Auditing privacy risk took %0.1f seconds", time.time() - baseline_time
#     )

# # Get average audit results across all experiments
# if len(target_model_indices) > 1:
#     get_average_audit_results(
#         directories["report_dir"], mia_score_list, membership_list, logger
#     )

#logger.info("Total runtime: %0.5f seconds", time.time() - start_time)

## Save protected attributes

In [ ]:
# save_path = directories['report_dir'] + "/pro_attributes.npy"
# pro_attributes = []
# for idx, (_, _, z) in enumerate(dataset):
#     pro_attributes.append((idx, z))
# pro_attributes = np.array(pro_attributes)
# np.save(save_path, pro_attributes)

## Audit by epsilon

In [ ]:
all_eps = exp_estimated_epsilon(
    f"{directories['report_dir']}/{configs["audit"]["algorithm"].lower()}", 
    mia_score_list, 
    membership_list, 
    dataset, 
    configs, 
    logger)

In [10]:
#算auc
from sklearn.metrics import roc_curve, auc
all_auc = {i:[] for i in range(10)}
for i, (s, m) in enumerate(zip(mia_score_list, membership_list)):
    m = m.astype(int)
    fpr_list, tpr_list, _ = roc_curve(m, s)
    roc_auc = auc(fpr_list, tpr_list)
    all_auc[dataset[i][1]].append(roc_auc)
    # _, emp_eps_loss = compute_eps_lower_from_mia(s, m, 0.05, configs["train"]["delta"], 'GDP', n_procs=32)
    # all_eps[dataset[i][1]].append(emp_eps_loss)

#每一类的平均auc
for i in range(10):
    print(np.mean(all_auc[i]))

0.5646255319148936
0.5432588809946715
0.601818968968969
0.6027196376101861
0.5981981025641026
0.6314512195121951
0.5766478636826042
0.5787186660268714
0.6233127692307693
0.6105920020120723


In [ ]:
#每一类的平均
for i in range(10):
    print(f"{i}:{np.mean(all_eps[i])}, {np.std(all_eps[i])}")

0:0.42922727343179023
1:0.28738560845276107
2:0.6747254973767883
3:0.6408867989409675
4:0.5466770951671491
5:0.7702012655963631
6:0.47327895370018097
7:0.475372824827916
8:0.7973298190483309
9:0.6153794759383451
